# Given

A sample of categorical data:

|   Ear Shape | Face Shape | Whiskers |   Cat (1-yes, 0-no)  |
|:---------:|:-----------:|:---------:|:------:|
|   Pointy   |   Round     |  Present  |    1   |
|   Floppy   |  Not Round  |  Present  |    1   |
|   Floppy   |  Round      |  Absent   |    0   |
|   Pointy   |  Not Round  |  Present  |    0   |


# Find 

one hot encode the features and build the decision tree:
- find the feature to split (entropy)
    - define impurity
    - calculate information gain
- split recursively
- stop splitting
    - when node is 100% purity
    - maximum depth of tree exceeds the defined level
    - improvements in information gain is too small
    - number of examples in the node is lower than predefined threshold

# Solution

In [371]:
import numpy as np

In [372]:
x = np.array([[1, 1, 1], [0, 0, 1], [0, 1, 0], [1, 0, 1], [1, 1, 1], [1, 1, 0], [0, 0, 0], [1, 1, 0], [0, 1, 0], [0, 1, 0]])
y = np.array([1, 1, 0, 0, 1, 1, 0, 1, 0, 0])

x.shape, y.shape

((10, 3), (10,))

Entropy 

* in general for multi-class problems:
$$H(X) = -\sum_{i=1}^{n} p(x_i) \log_{2}(p(x_i))$$

* in binary classification simplifies to:
$$H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)$$
where $p_1$ is the quantity of dogs (1s) in the dataset, and $p_0 = 1-p_1$ is the probability of cats

In [373]:
def calculate_entropy(branch):
    # In - inner branch with values inside = [0, 1, 1, 0, 1]
    # Out - 
    #   entropy = dirtiness of a dataset. 0 - ideal, 1 - bad
    #   p - weight

    p = np.sum(branch) / len(branch)
    
    if p==1 or p==0:
        entropy = 0
    else:
        entropy = - p * np.log2(p) - (1-p) * np.log2(1-p)
    
    return entropy, p

Information gain

* in mgeneral for multi-class:

    $$\text{Information Gain}(S, A) = \text{Entropy}(S) - \sum_{v \in \text{Values}(A)} \frac{|S_v|}{|S|} \cdot \text{Entropy}(S_v)$$

    , where 
    * $S$ = dataset, 
    * $A$ = attribute, 
    * $|S|$ - total number of instances in the dataset $S$,
    * $|S_v|$ - subset of instances where attribute $A$ has value $v$
    * $\text{Values}(A)$ - possible values of attribute A 
    
<p>

* in binary classiciation simplifies to:

$$\text{Information Gain} = H(p_{root})- \left(w_{\text{left}}\cdot H\left(p_1^\text{left}\right) + w_{\text{right}}\cdot H\left(p_1^\text{right}\right)\right),$$

In [374]:
def calculate_ig(parent, children = []):

    H_root, _ = calculate_entropy(parent)

    sum = 0

    for child in children:

        H, w = calculate_entropy(child)
        sum += H * w

    IG = H_root - sum
    
    return IG

In [375]:
calculate_ig(y, [y[x[:,0]==0],y[x[:,0]==1]])

0.2780719051126377

In [423]:
def choose_best_feature(x, y, processed_nodes = []):

    ig = []

    for feature_id in range(x.shape[1]):
        if feature_id not in processed_nodes:
            left_subset = y[x[:,feature_id]==0]
            right_subset = y[x[:,feature_id]==1]
            feature_ig = calculate_ig(y, [left_subset, right_subset])
            ig.append(feature_ig)
        else:
            ig.append(0)

    print(ig)

    return(np.argmax(ig))


In [377]:
def split_feature(x, feature_to_split_on):
    # input - x - dataset
    #   feature = 2 (ex)
    # output - tbd

    feature_idx = {}

    for feature in x[:,feature_to_split_on]:
        idx = np.where(x[:,feature_to_split_on]==feature)
        feature_idx[feature] = idx

    return feature_idx

In [378]:
split_feature(x,0)

{1: (array([0, 3, 4, 5, 7]),), 0: (array([1, 2, 6, 8, 9]),)}

Tree and node classes

In [419]:
class TreeNode():

    left_feature_id = -1
    right_feature_id = -1

    rows_idx = []
    path = []

    left_feature = TreeNode()
    right_feature = TreeNode()

    def __repr__(self):
        attrs = vars(self)
        return ', '.join(f"{key}={value}" for key, value in attrs.items())


class Tree():
    root_feature = -1
    node = TreeNode()

In [424]:
def process(current = TreeNode(), x = [[]], y = [], path = []):

    if len(path) > 1:
        print("bottom")
        return 0

    # calculate left and right split based on the parent

    parent = path[-1]

    x_left = x[x[:,parent]==0]
    x_right = x[x[:,parent]==1]

    y_left = y[x[:,parent]==0]
    y_right = y[x[:,parent]==1]

    best_split_left = choose_best_feature(x_left, y_left, processed_nodes=path)
    best_split_right = choose_best_feature(x_right, y_right, processed_nodes=path)

    # call function again for each of the splits

    current.left_feature_id = best_split_left
    current.right_feature_id = best_split_right
    current.path = path
    current.rows_idx = "all"

    print(current)

    process(current.left_feature, x_left, y_left, path + [best_split_left])
    process(current.left_feature, x_right, y_right, path + [best_split_right])


Calculate root

In [425]:
tree = Tree()
tree.root_feature = choose_best_feature(x, y)


process(tree.node, x, y, path=[tree.root_feature,])

[0.2780719051126377, 0.13091388234321688, 0.08544279530415388]
[0, 0.22192809488736231, 0.7219280948873623]
[0, 0.7219280948873623, 0.10973087218436928]
left_feature_id=2, right_feature_id=1, path=[0], rows_idx=all
bottom
bottom


In [405]:
print(tree.node)

Fork

In [263]:
def build_branch(parent):

    """
    parent = [his_x_idx, his_y_idx, processed_features]
    child = {feature_id: 2, true: [new_x, new_y, processed_features], false: [new_x, new_y, processed_features]}
    """

    x = parent[0]
    y = parent[1]
    processed_features = parent[2]

    # list features available [1,2,3]
        # calculate ig for each feature

    ig = []

    for feature in range(x.shape[1]):
        if feature not in processed_features:
            left_branch = y[x[:,feature]==0]
            right_branch = y[x[:,feature]==1]
            ig.append(calculate_ig(y, left_branch, right_branch))

    if np.sum(ig) == 0:
        print("all features processed")
        return 0

    # pick max ig and add to the tree [3]

    best_pick = np.argmax(ig)

    print(f"best pick = {best_pick}, ig = {ig}")

    # split the features in left/right [x_left, x_right]

    _,  = split_feature(x, best_pick)

    fork = [left_idx, right_idx]

    # add to the tree [x_left ig][x_right ig]
        
    return fork

In [265]:
decisionTree = tree()

processed_nodes = []

best_pick, left_idx, right_idx = build_branch([x, y])
processed_nodes.append(best_pick)
decisionTree[best_pick]['no']=[left_idx, right_idx]
decisionTree[best_pick]['yes']=[left_idx, right_idx]

best pick = 0, ig = [0.2780719051126377, 0.13091388234321688, 0.08544279530415388]


In [266]:

def scan_tree_inside(decisionTree):

    for key, value in decisionTree.items():
        if type(value) == collections.defaultdict:
            scan_tree_inside(value)
        else:
            print(f"{key}:{value}")

            best_pick, x_left, x_right, y_left, y_right = build_branch([x, y])
            decisionTree[key][best_pick]['no'] = [x_left, y_left]
            decisionTree[key][best_pick]['yes'] = [x_right, y_right]

In [267]:
scan_tree_inside(decisionTree)

no:[(array([1, 2, 6, 8, 9]),), (array([0, 3, 4, 5, 7]),)]
best pick = 0, ig = [0.2780719051126377, 0.13091388234321688, 0.08544279530415388]


ValueError: not enough values to unpack (expected 5, got 3)

In [268]:
decisionTree

defaultdict(<function __main__.tree()>,
            {0: defaultdict(<function __main__.tree()>,
                         {'no': [(array([1, 2, 6, 8, 9]),),
                           (array([0, 3, 4, 5, 7]),)],
                          'yes': [(array([1, 2, 6, 8, 9]),),
                           (array([0, 3, 4, 5, 7]),)]})})